# Extraction of Spindles from Neocortex recordings

Restarting from LFPwake0 and LFPwakeremoved.

LFPwakeremoved will be used to determined signal variance for threshold adjustement. 

LFPwake0 will be used for time determination. 

## Load LFP and packages

In [68]:
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
from scipy import fftpack
import pandas as pd

%matplotlib widget

EMGboolean = pd.read_pickle('EMGframeBoolean_2.pkl')
LFPwakeremoved = np.load('LFPwakeremoved_2.npy', mmap_mode= 'r')
All = np.load('RawDataChannelExtractedDS_2.npy', mmap_mode= 'r')
PFC = All[:, 1]
PFCwakeremoved = LFPwakeremoved[:,0]
S1 = All[:, 2]
S1wakeremoved = LFPwakeremoved[:,1]

# Band pass filter
        Spindles: 8-20 Hz

In [69]:
# Filtre parameter:
f_lowcut = 8.
f_hicut = 20.
fs = 1000
nyq = 0.5 * fs
N = 4                 # Filtre order
Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction

# Filtering:
b, a = signal.butter(N, Wn, 'band')
filt_PFC = signal.filtfilt(b, a, PFC)
filt_PFCwakeremoved = signal.filtfilt(b, a, PFCwakeremoved)

filt_S1 = signal.filtfilt(b, a, S1)
filt_S1wakeremoved = signal.filtfilt(b, a, S1wakeremoved)

# Plot
# times = np.arange(0, PFC.size/fs, 1./fs)
# timesmin = np.arange(0, PFC.size/fs/60, 1./fs/60)
# fig, ax = plt.subplots()
# ax.plot(timesmin, filt_S1)

## Continuous Wavelet Transform and projection calculation

First on signal with no wake time to determine sd of signal

In [70]:
# Parameter and computation of CWT
w = 10.
freq = np.linspace(8, 20, 24)
widths = w*fs / (2*freq*np.pi)
PFCNWcwt = signal.cwt(filt_PFCwakeremoved, signal.morlet2, widths, w=w)
S1NWcwt = signal.cwt(filt_S1wakeremoved, signal.morlet2, widths, w=w)

# Projection calculation PFC
absPFCNWcwt = np.absolute(PFCNWcwt)
proj_PFCNWcwt = np.sum(absPFCNWcwt, axis = 0)/24
sdproj_PFCcwt = np.std(proj_PFCNWcwt)
sd7proj_PFCcwt = sdproj_PFCcwt*7

# Projection calculation S1
absS1NWcwt = np.absolute(S1NWcwt)
proj_S1NWcwt = np.sum(absS1NWcwt, axis = 0)/24
sdproj_S1cwt = np.std(proj_S1NWcwt)
sd7proj_S1cwt = sdproj_S1cwt*7


Second on the signal for which wake times have been zeroed

In [71]:
#####################################
########        PFC         #########
#####################################
# Conservative boolean filtering of PFC filtered signal
BooleanCons = EMGboolean['BooleanConservative']
fPFCwake0C = filt_PFC.copy()
fPFCwake0C[BooleanCons] = 0
PFCwake0C = PFC.copy()
PFCwake0C[BooleanCons] = 0
# Liberal boolean filtering of PFC filtered signal
BooleanLib = EMGboolean['BooleanLiberal']
fPFCwake0L = filt_PFC.copy()
fPFCwake0L[BooleanLib] = 0
PFCwake0L = PFC.copy()
PFCwake0L[BooleanLib] = 0

# Computation of CWT
PFCcwtWake0cons = signal.cwt(fPFCwake0C, signal.morlet2, widths, w=w)
PFCcwtWake0lib = signal.cwt(fPFCwake0L, signal.morlet2, widths, w=w)

# Projection calculation
absPFCW0Ccwt = np.absolute(PFCcwtWake0cons)
proj_PFCW0Ccwt = np.sum(absPFCW0Ccwt, axis = 0)/24
absPFCW0Lcwt = np.absolute(PFCcwtWake0lib)
proj_PFCW0Lcwt = np.sum(absPFCW0Lcwt, axis = 0)/24

#####################################
########         S1         #########
#####################################
# Conservative boolean filtering of S1 filtered signal
BooleanCons = EMGboolean['BooleanConservative']
fS1wake0C = filt_S1.copy()
fS1wake0C[BooleanCons] = 0
S1wake0C = S1.copy()
S1wake0C[BooleanCons] = 0
# Liberal boolean filtering of S1 filtered signal
BooleanLib = EMGboolean['BooleanLiberal']
fS1wake0L = filt_S1.copy()
fS1wake0L[BooleanLib] = 0
S1wake0L = S1.copy()
S1wake0L[BooleanLib] = 0

# Computation of CWT
S1cwtWake0cons = signal.cwt(fS1wake0C, signal.morlet2, widths, w=w)
S1cwtWake0lib = signal.cwt(fS1wake0L, signal.morlet2, widths, w=w)

# Projection calculation
absS1W0Ccwt = np.absolute(S1cwtWake0cons)
proj_S1W0Ccwt = np.sum(absS1W0Ccwt, axis = 0)/24
absS1W0Lcwt = np.absolute(S1cwtWake0lib)
proj_S1W0Lcwt = np.sum(absS1W0Lcwt, axis = 0)/24

### Display subset 

Not necessary cell to run

In [ ]:
# Defining subset
start = 00000
end = 800000

times = np.arange(0, PFC.size/fs, 1./fs)
tt = times[start:end]
Cortext = PFCwake0C[start:end]/10
#Cortexcwtt = PFCNWcwt[:, start:end]
proj_Cortexcwtt = proj_PFCW0Ccwt[start:end]

plt.close('all')
plt.axhline(sdproj_PFCcwt, color='r') # horizontal
plt.axhline(sd7proj_PFCcwt, color='g') # horizontal
plt.plot(tt, Cortext)
plt.plot(tt, proj_Cortexcwtt)
plt.show()

## WARNING: Plot only short subsets (~ 10 s), too memory consuming otherwise
#plt.pcolormesh(tt, freq, np.abs(PFCcwtt), cmap='viridis', shading='gouraud')
#plt.plot(tt, PFCt)
#plt.show()


Display with ephyviewer, not needed

In [11]:
from ephyviewer import mkQApp, MainViewer, TraceViewer

combined = np.stack([PFC, filt_PFC, proj_PFCW0Ccwt, proj_PFCW0Lcwt, S1, filt_S1, proj_S1W0Ccwt, proj_S1W0Lcwt], axis = 1)

app = mkQApp()

sample_rate = 1000.
t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer()
view1 = TraceViewer.from_numpy(combined, sample_rate, t_start, 'Signals')
win.add_view(view1)

#Parameters can be set in script
#view1.params['scale_mode'] = 'same_for_all'
view1.params['display_labels'] = True
#And also parameters for each channel
view1.by_channel_params['ch0', 'color'] = '#ffffff'
view1.by_channel_params['ch1', 'color'] = '#0055ff'
view1.by_channel_params['ch2', 'color'] = '#ff5500'
view1.by_channel_params['ch3', 'color'] = '#ffffff'
view1.by_channel_params['ch4', 'color'] = '#ffffff'
view1.by_channel_params['ch5', 'color'] = '#0055ff'
view1.by_channel_params['ch6', 'color'] = '#ff5500'
view1.by_channel_params['ch7', 'color'] = '#ffffff'

view1.by_channel_params['ch0', 'gain'] = 0.05
view1.by_channel_params['ch1', 'gain'] = 0.1
view1.by_channel_params['ch2', 'gain'] = 0.1
view1.by_channel_params['ch3', 'gain'] = 0.1
view1.by_channel_params['ch4', 'gain'] = 0.05
view1.by_channel_params['ch5', 'gain'] = 0.1
view1.by_channel_params['ch6', 'gain'] = 0.1
view1.by_channel_params['ch7', 'gain'] = 0.1

view1.by_channel_params['ch0', 'offset'] = 200
view1.by_channel_params['ch1', 'offset'] = 0
view1.by_channel_params['ch2', 'offset'] = 0
view1.by_channel_params['ch3', 'offset'] = 0
view1.by_channel_params['ch4', 'offset'] = -200
view1.by_channel_params['ch5', 'offset'] = -400
view1.by_channel_params['ch6', 'offset'] = -400
view1.by_channel_params['ch7', 'offset'] = -400


#Run
win.show()
app.exec_()

0

## Extracting Spindles and determining main properties 

First extraction of spindle peaks, initiation, end and width

In [79]:
from scipy.signal import find_peaks
from scipy.signal import chirp, find_peaks, peak_widths

cortex = S1
proj_cortex = proj_S1W0Lcwt
proj_cortexC = proj_S1W0Ccwt
sd_proj_cortex = sd7proj_S1cwt
lib_wake0_cortex = S1cwtWake0lib
filt_cortex = filt_S1

# 7 sd threshold
peaks, properties = find_peaks(proj_cortex, prominence=1, width=200, height=sd_proj_cortex)
properties["prominences"], properties["widths"]

# Spindles boundaries taken at 70% from peak of intensity. This means that the spindles with small amplitude will be longer than the big ones.
results_width = peak_widths(proj_cortex, peaks, rel_height=0.6)

# Organise results in numpy array
peaks2 = peaks.reshape(len(peaks),1)
npresults_width = np.array(results_width).reshape(4,-1)
Spindle_prop = np.append(peaks2, results_width).reshape(5,len(peaks2)).round()
peaks.shape

(5,)

Second extraction of main frequency and power 

In [80]:
projMaxP_cwtmg = np.max(lib_wake0_cortex, axis = 0)
projMaxF_cwtmg = np.argmax(lib_wake0_cortex, axis = 0)/2 + 8
projMaxP_cwtmg.shape

nb_Spindles = np.arange(0,len(peaks),1)
data = np.zeros((len(peaks),4))

for tt in nb_Spindles:
    Spindle_start = int(Spindle_prop[3,tt])
    Spindle_stop = int(Spindle_prop[4,tt])
    Spindle_MaxP = projMaxP_cwtmg[Spindle_start:Spindle_stop]
    Spindle_MaxF = projMaxF_cwtmg[Spindle_start:Spindle_stop]
    data[tt, 0] = max(Spindle_MaxF).round()
    data[tt, 1] = max(Spindle_MaxP).round()
    data[tt, 2] = round(sum(Spindle_MaxF)/len(Spindle_MaxF))
    data[tt, 3] = round(sum(Spindle_MaxP)/len(Spindle_MaxP))

param_Spindle = pd.DataFrame(data, columns = ['Max freq', 'Max int', 'Avg freq', 'Avg int'])
tSpindle_prop = Spindle_prop.transpose()
pd_prop_Spindle = pd.DataFrame(tSpindle_prop, columns = ['peak time', 'Duration', 'peak amp', 'start time', 'end time'])
All_Spindle = pd.concat([pd_prop_Spindle, param_Spindle], axis=1)

/var/folders/jq/nz6nvbd54ns2lkjvj31_7_4r0000gn/T/ipykernel_15385/1927274547.py:14: ComplexWarning: Casting complex values to real discards the imaginary part
  data[tt, 1] = max(Spindle_MaxP).round()
/var/folders/jq/nz6nvbd54ns2lkjvj31_7_4r0000gn/T/ipykernel_15385/1927274547.py:16: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  data[tt, 3] = round(sum(Spindle_MaxP)/len(Spindle_MaxP))
/var/folders/jq/nz6nvbd54ns2lkjvj31_7_4r0000gn/T/ipykernel_15385/1927274547.py:16: ComplexWarning: Casting complex values to real discards the imaginary part
  data[tt, 3] = round(sum(Spindle_MaxP)/len(Spindle_MaxP))


### Store the results in All_Spindle_prop pd dataframe and save as pkl/csv for post processing.

End of Notebook. 

In [81]:
All_Spindle.to_pickle('Spindleproperties_S1_2.pkl')
All_Spindle.to_csv('Spindleproperties_S1_2.csv', sep = ',')

# if done and no intention to display for assessment
#%reset
#plt.close('all')

### Display

ephys viewer to check Spindle detection

In [82]:
from ephyviewer import mkQApp, MainViewer, TraceViewer
from ephyviewer import AnalogSignalSourceWithScatter
import ephyviewer

app = mkQApp()

sample_rate = 1000.
t_start = 0.

combined = np.stack([cortex, filt_cortex, proj_cortexC, proj_cortex], axis = 1)

Spindle_peak = peaks
Spindle_start = Spindle_prop[3,:].astype(int)
Spindle_end = Spindle_prop[4,:].astype(int)

#create 2 familly scatters from theses 2 indexes
scatter_indexes = {0: Spindle_peak, 1: Spindle_start, 2: Spindle_end}
#and asign them to some channels each
scatter_channels = {0: [1, 2], 1: [0, 1], 2: [0, 1]}
source = AnalogSignalSourceWithScatter(combined, sample_rate, t_start, scatter_indexes, scatter_channels)

#Create the main window that can contain several viewers
win = MainViewer(debug=True, show_auto_scale=True)

#create a viewer for signal with TraceViewer
#connected to the signal source
view1 = TraceViewer(source=source)

#Parameters can be set in script
#view1.params['scale_mode'] = 'same_for_all'
view1.params['display_labels'] = True
#And also parameters for each channel
view1.by_channel_params['ch0', 'color'] = '#ffffff'
view1.by_channel_params['ch1', 'color'] = '#0055ff'
view1.by_channel_params['ch2', 'color'] = '#ff5500'
view1.by_channel_params['ch3', 'color'] = '#ffffff'

view1.by_channel_params['ch0', 'gain'] = 0.003
view1.by_channel_params['ch1', 'gain'] = 0.004
view1.by_channel_params['ch2', 'gain'] = 0.002
view1.by_channel_params['ch3', 'gain'] = 0.002


view1.by_channel_params['ch0', 'offset'] = 6
view1.by_channel_params['ch1', 'offset'] = -1
view1.by_channel_params['ch2', 'offset'] = -1
view1.by_channel_params['ch3', 'offset'] = -1

#put this viewer in the main window
win.add_view(view1)

#Run
win.show()
app.exec_()

debug True
QT_MODE PyQt5
refresh duration for  0.00024509429931640625 s


qt.qpa.backingstore: Back buffer dpr of 2 doesn't match <_NSViewBackingLayer: 0x7fdc643f7810> contents scale of 1 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <_NSViewBackingLayer: 0x7fdc643f7810> contents scale of 2 - updating layer to match.


refresh duration for  0.00023508071899414062 s
refresh duration for  0.0002319812774658203 s
refresh duration for  0.00018596649169921875 s
refresh duration for  0.00016379356384277344 s
refresh duration for  0.00017571449279785156 s
refresh duration for  0.0001659393310546875 s
refresh duration for  0.0002219676971435547 s
refresh duration for  0.00021409988403320312 s
refresh duration for  0.0002498626708984375 s
refresh duration for  0.00024008750915527344 s
refresh duration for  0.0002586841583251953 s
refresh duration for  0.0002751350402832031 s
refresh duration for  0.00018310546875 s
refresh duration for  0.0001690387725830078 s
refresh duration for  0.0002281665802001953 s
refresh duration for  0.0001659393310546875 s
refresh duration for  0.00023293495178222656 s
refresh duration for  0.00023031234741210938 s
refresh duration for  0.0002589225769042969 s
refresh duration for  0.00024700164794921875 s
refresh duration for  0.0002338886260986328 s
refresh duration for  0.000580

0